In [1]:
!pip install pymupdf
!pip install langchain
!pip install lancedb
!pip install -U langchain_ollama

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 27.9 MB/s eta 0:00:00m eta 0:00:010:00:01


In [1]:
import fitz
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [2]:
from langchain_community.embeddings import OllamaEmbeddings
from langchain.vectorstores import LanceDB
from langchain.chains import RetrievalQA
from langchain_ollama import OllamaEmbeddings
from langchain_ollama import OllamaLLM

In [3]:
import lancedb

/home/u/RAG/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
doc = fitz.open("DS.pdf")

In [5]:
texts, metadatas = [], []


In [6]:
for page_num in range(len(doc)):
    page = doc.load_page(page_num)
    texts.append(page.get_text())
    metadatas.append(doc.metadata)

MuPDF error: syntax error: invalid key in dict

MuPDF error: format error: non-page object in page tree

MuPDF error: syntax error: invalid key in dict

MuPDF error: syntax error: invalid key in dict

MuPDF error: syntax error: invalid key in dict

MuPDF error: syntax error: invalid key in dict

MuPDF error: syntax error: invalid key in dict

MuPDF error: syntax error: invalid key in dict

MuPDF error: syntax error: invalid key in dict

MuPDF error: syntax error: invalid key in dict

MuPDF error: syntax error: invalid key in dict

MuPDF error: syntax error: invalid key in dict

MuPDF error: syntax error: invalid key in dict

MuPDF error: syntax error: invalid key in dict

MuPDF error: syntax error: invalid key in dict

MuPDF error: syntax error: invalid key in dict

MuPDF error: syntax error: invalid key in dict

MuPDF error: syntax error: invalid key in dict

MuPDF error: syntax error: invalid key in dict

MuPDF error: syntax error: invalid key in dict

MuPDF error: syntax error: inva

In [7]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
document=text_splitter.create_documents(texts,metadatas=metadatas)

In [8]:
embeddings = OllamaEmbeddings(model="deepseek-r1:14b")

In [9]:
db = lancedb.connect("/tmp/lancedb")

In [10]:
vectorstore = LanceDB.from_documents(document, embeddings, connection=db)

In [11]:
llm = OllamaLLM(model="deepseek-r1:14b")

In [12]:
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 4})

In [13]:
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=retriever)

In [14]:
query = "What are the main topics discussed in the documents?"

In [23]:
result = qa.invoke(query)


In [24]:
print(result)

{'query': 'What are the main topics discussed in the documents?', 'result': '<think>\n嗯，我现在需要帮助用户分析他们提供的文档内容。首先，我看到了关于Deepseek的一些信息，包括它的定义、功能以及使用方法。这可能是一个AI工具，用于各种任务。接着，有一大块关于创意概念设计的内容，里面提到了理论和方法层面的多个要点，比如跨领域联想指令、品牌DNA融入等等。\n\n用户还提供了一个具体的营销创意方案的要求，需要结合两个看似不相关的领域，融入品牌价值，设计记忆点、情感触发元素等，并且考虑传播潜力和可持续性。此外，还有关于如何从入门到精通使用AI的建议，提到了TASTE和ALIGN框架，以及一些提示词的例子。\n\n现在用户的问题是询问文档中讨论的主要主题是什么。我需要整理这些内容，找出核心话题。首先，创意概念设计是一个主要部分，涉及营销策略和品牌推广。其次，Deepseek作为一个工具，可能与AI相关，用于执行各种任务。然后，如何有效使用AI的建议也是一个关键点。\n\n接下来，用户提供的示例显示他们希望生成创意方案，结合不同领域，并且具备传播潜力。这说明文档不仅讨论了理论，还提供了实际应用的方法和框架。同时，关于AI使用的部分强调了在竞争环境中脱颖而出的方法，这也是一个重要的主题。\n\n综上所述，文档主要围绕三个核心主题：Deepseek的功能与使用、创意概念设计的策略，以及高效利用AI的技巧。这些内容结合起来，展示了如何通过创新思维和先进工具来提升品牌营销和AI应用的效果。\n</think>\n\nThe main topics discussed in the documents are:\n\n1. **Creative Concept Design**: This includes strategies for designing innovative marketing campaigns, incorporating elements from unrelated fields, brand value integration, memorable slogans, emotional triggers, and leveraging current trends.